# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

import seaborn as sns
import matplotlib.pyplot as plt

# from sklearn.model_selection import train_test_split
# from sklearn import metrics
# from sklearn.ensemble import RandomForestClassifier

# from scipy.stats import ks_2samp
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

%matplotlib inline

In [4]:
df = pd.read_csv('previsao_de_renda.csv')
df

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,2016-03-01,16006,F,False,True,0,Empresário,Secundário,Solteiro,Casa,48,13.887671,1.0,7990.58
14996,14996,2016-03-01,3722,F,False,True,0,Pensionista,Superior completo,Solteiro,Casa,57,NaN,1.0,10093.45
14997,14997,2016-03-01,6194,F,True,True,0,Assalariado,Superior completo,Casado,Casa,45,7.832877,2.0,604.82
14998,14998,2016-03-01,4922,M,True,False,0,Empresário,Superior completo,Casado,Casa,36,4.298630,2.0,3352.27


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [11]:
# 1: Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
X = df.drop(columns = ["renda", "data_ref", "estado_civil", "tipo_residencia", "tipo_renda","educacao", "posse_de_veiculo", "posse_de_imovel","sexo"])
X = pd.get_dummies(X, drop_first = True)
print(X.info())
X.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   id_cliente             15000 non-null  int64  
 2   qtd_filhos             15000 non-null  int64  
 3   idade                  15000 non-null  int64  
 4   tempo_emprego          12427 non-null  float64
 5   qt_pessoas_residencia  15000 non-null  float64
dtypes: float64(2), int64(4)
memory usage: 703.3 KB
None


,Unnamed: 0,id_cliente,qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia
0,0,15056,0,26,6.602740,1.0
1,1,9968,0,28,7.183562,2.0
2,2,4312,0,35,0.838356,2.0
3,3,10639,1,30,4.846575,3.0
4,4,7064,0,33,4.293151,1.0


In [19]:
# 1:

novo_data = pd.DataFrame(df, columns=["renda", "id_cliente", "idade", "qtd_filhos", "qt_pessoas_residencia"])
novo_data

,renda,id_cliente,idade,qtd_filhos,qt_pessoas_residencia
0,8060.34,15056,26,0,1.0
1,1852.15,9968,28,0,2.0
2,2253.89,4312,35,0,2.0
3,6600.77,10639,30,1,3.0
4,6475.97,7064,33,0,1.0
...,...,...,...,...,...
14995,7990.58,16006,48,0,1.0
14996,10093.45,3722,57,0,1.0
14997,604.82,6194,45,0,2.0
14998,3352.27,4922,36,0,2.0


In [27]:
# 1:

X = df[["id_cliente", "idade", "qtd_filhos", "qt_pessoas_residencia"]]
X

,id_cliente,idade,qtd_filhos,qt_pessoas_residencia
0,15056,26,0,1.0
1,9968,28,0,2.0
2,4312,35,0,2.0
3,10639,30,1,3.0
4,7064,33,0,1.0
...,...,...,...,...
14995,16006,48,0,1.0
14996,3722,57,0,1.0
14997,6194,45,0,2.0
14998,4922,36,0,2.0


In [28]:
# 1:

train_test_split(X)

[       id_cliente  idade  qtd_filhos  qt_pessoas_residencia
 5024         5621     39           1                    2.0
 12867        3648     66           0                    2.0
 10638        6979     32           0                    2.0
 2228         9168     51           1                    3.0
 12710        3041     52           0                    2.0
 ...           ...    ...         ...                    ...
 5783         2662     42           0                    2.0
 12866       11156     38           2                    4.0
 11234        5616     35           0                    2.0
 3083         6681     38           1                    3.0
 4967         7871     25           0                    1.0
 
 [11250 rows x 4 columns],
        id_cliente  idade  qtd_filhos  qt_pessoas_residencia
 9138         5164     58           0                    2.0
 8038        12046     47           0                    2.0
 11978       15153     34           2                   

In [29]:
# 1:

y = df[["renda"]]
y

,renda
0,8060.34
1,1852.15
2,2253.89
3,6600.77
4,6475.97
...,...
14995,7990.58
14996,10093.45
14997,604.82
14998,3352.27


In [44]:
# 2:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [47]:
# 2:
ridge_model = Ridge(alpha=0.1)
ridge_model.fit(X_train, y_train)
y_pred = ridge_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

print(f"MSE: {mse}")
print(f"Coeficientes: {ridge_model.coef_}")

MSE: 48740286.80161255
Coeficientes: [[ 2.87050323e-03  3.18514249e+01 -6.24092185e+02  9.98632795e+02]]


In [46]:
# 2:
ridge_model_0 = Ridge(alpha=0)
ridge_model_0.fit(X_train, y_train)
y_pred = ridge_model_0.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

print(f"MSE: {mse}")
print(f"Coeficientes: {ridge_model_0.coef_}")

MSE: 48740290.167965636
Coeficientes: [[ 2.87035010e-03  3.18513080e+01 -6.24189694e+02  9.98713448e+02]]


In [48]:
# 2:
ridge_model = Ridge(alpha=0.001)
ridge_model.fit(X_train, y_train)
y_pred = ridge_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

print(f"MSE: {mse}")
print(f"Coeficientes: {ridge_model.coef_}")

MSE: 48740290.13428623
Coeficientes: [[ 2.87035163e-03  3.18513092e+01 -6.24188718e+02  9.98712641e+02]]


In [49]:
# 2:
ridge_model = Ridge(alpha=0.005)
ridge_model.fit(X_train, y_train)
y_pred = ridge_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

print(f"MSE: {mse}")
print(f"Coeficientes: {ridge_model.coef_}")

MSE: 48740289.99957184
Coeficientes: [[ 2.87035776e-03  3.18513139e+01 -6.24184818e+02  9.98709415e+02]]


In [50]:
# 2:
ridge_model = Ridge(alpha=0.05)
ridge_model.fit(X_train, y_train)
y_pred = ridge_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

print(f"MSE: {mse}")
print(f"Coeficientes: {ridge_model.coef_}")

MSE: 48740288.484388374
Coeficientes: [[ 2.87042667e-03  3.18513665e+01 -6.24140936e+02  9.98673119e+02]]


In [53]:
# 2:
ridge_model = Ridge(alpha=0.01)
ridge_model.fit(X_train, y_train)
y_pred = ridge_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

print(f"MSE: {mse}")
print(f"Coeficientes: {ridge_model.coef_}")

MSE: 48740289.83118606
Coeficientes: [[ 2.87036541e-03  3.18513197e+01 -6.24179942e+02  9.98705382e+02]]


In [52]:
# 3: 
modelo = "renda ~ id_cliente + idade + qtd_filhos + qt_pessoas_residencia"
md = smf.ols(modelo, data = df)
reg = md.fit_regularized(method = "elastic_net"
                        , refit = True
                        , L1_wt = 1
                        , alpha = 0)
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     13.93
Date:                Fri, 16 Feb 2024   Prob (F-statistic):           1.31e-13
Time:                        01:57:17   Log-Likelihood:            -1.5655e+05
No. Observations:               15000   AIC:                         3.131e+05
Df Residuals:                   14995   BIC:                         3.132e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept              2630.2191    442.985      5.937      0.000    1761.914    3498.524
id_cliente               -0.0161      0.014     -1.146      0.252      -0.044       0.011
idade                    29.1542      6.419      4.542      0.000      16.572      41.736
qtd_filhos             -580.9569    199.538     -2.912      0.004    -972.076    -189.838
qt_pessoas_residencia   984.7392    162.404      6.064      0.000     666.408    1303.070
==============================================================================
Omnibus:                    21437.439   Durbin-Watson:                   2.026
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         11006137.536
Skew:                           8.364   Prob(JB):                         0.00
Kurtosis:                     134.643   Cond. No.                     6.67e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.67e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [54]:
# 3: 
modelo = "renda ~ id_cliente + idade + qtd_filhos + qt_pessoas_residencia"
md = smf.ols(modelo, data = df)
reg = md.fit_regularized(method = "elastic_net"
                        , refit = True
                        , L1_wt = 1
                        , alpha = 0.01)
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     13.93
Date:                Fri, 16 Feb 2024   Prob (F-statistic):           1.31e-13
Time:                        01:57:56   Log-Likelihood:            -1.5655e+05
No. Observations:               15000   AIC:                         3.131e+05
Df Residuals:                   14995   BIC:                         3.132e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept              2630.2191    442.985      5.937      0.000    1761.914    3498.524
id_cliente               -0.0161      0.014     -1.146      0.252      -0.044       0.011
idade                    29.1542      6.419      4.542      0.000      16.572      41.736
qtd_filhos             -580.9569    199.538     -2.912      0.004    -972.076    -189.838
qt_pessoas_residencia   984.7392    162.404      6.064      0.000     666.408    1303.070
==============================================================================
Omnibus:                    21437.439   Durbin-Watson:                   2.026
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         11006137.536
Skew:                           8.364   Prob(JB):                         0.00
Kurtosis:                     134.643   Cond. No.                     6.67e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.67e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [55]:
# 3: 
modelo = "renda ~ id_cliente + idade + qtd_filhos + qt_pessoas_residencia"
md = smf.ols(modelo, data = df)
reg = md.fit_regularized(method = "elastic_net"
                        , refit = True
                        , L1_wt = 1
                        , alpha = 0.05)
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     13.93
Date:                Fri, 16 Feb 2024   Prob (F-statistic):           1.31e-13
Time:                        01:58:29   Log-Likelihood:            -1.5655e+05
No. Observations:               15000   AIC:                         3.131e+05
Df Residuals:                   14995   BIC:                         3.132e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept              2630.2191    442.985      5.937      0.000    1761.914    3498.524
id_cliente               -0.0161      0.014     -1.146      0.252      -0.044       0.011
idade                    29.1542      6.419      4.542      0.000      16.572      41.736
qtd_filhos             -580.9569    199.538     -2.912      0.004    -972.076    -189.838
qt_pessoas_residencia   984.7392    162.404      6.064      0.000     666.408    1303.070
==============================================================================
Omnibus:                    21437.439   Durbin-Watson:                   2.026
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         11006137.536
Skew:                           8.364   Prob(JB):                         0.00
Kurtosis:                     134.643   Cond. No.                     6.67e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.67e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [56]:
# 3: 
modelo = "renda ~ id_cliente + idade + qtd_filhos + qt_pessoas_residencia"
md = smf.ols(modelo, data = df)
reg = md.fit_regularized(method = "elastic_net"
                        , refit = True
                        , L1_wt = 1
                        , alpha = 0.005)
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     13.93
Date:                Fri, 16 Feb 2024   Prob (F-statistic):           1.31e-13
Time:                        01:58:57   Log-Likelihood:            -1.5655e+05
No. Observations:               15000   AIC:                         3.131e+05
Df Residuals:                   14995   BIC:                         3.132e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept              2630.2191    442.985      5.937      0.000    1761.914    3498.524
id_cliente               -0.0161      0.014     -1.146      0.252      -0.044       0.011
idade                    29.1542      6.419      4.542      0.000      16.572      41.736
qtd_filhos             -580.9569    199.538     -2.912      0.004    -972.076    -189.838
qt_pessoas_residencia   984.7392    162.404      6.064      0.000     666.408    1303.070
==============================================================================
Omnibus:                    21437.439   Durbin-Watson:                   2.026
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         11006137.536
Skew:                           8.364   Prob(JB):                         0.00
Kurtosis:                     134.643   Cond. No.                     6.67e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.67e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [57]:
# 3: 
modelo = "renda ~ id_cliente + idade + qtd_filhos + qt_pessoas_residencia"
md = smf.ols(modelo, data = df)
reg = md.fit_regularized(method = "elastic_net"
                        , refit = True
                        , L1_wt = 1
                        , alpha = 0.001)
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     13.93
Date:                Fri, 16 Feb 2024   Prob (F-statistic):           1.31e-13
Time:                        01:59:36   Log-Likelihood:            -1.5655e+05
No. Observations:               15000   AIC:                         3.131e+05
Df Residuals:                   14995   BIC:                         3.132e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept              2630.2191    442.985      5.937      0.000    1761.914    3498.524
id_cliente               -0.0161      0.014     -1.146      0.252      -0.044       0.011
idade                    29.1542      6.419      4.542      0.000      16.572      41.736
qtd_filhos             -580.9569    199.538     -2.912      0.004    -972.076    -189.838
qt_pessoas_residencia   984.7392    162.404      6.064      0.000     666.408    1303.070
==============================================================================
Omnibus:                    21437.439   Durbin-Watson:                   2.026
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         11006137.536
Skew:                           8.364   Prob(JB):                         0.00
Kurtosis:                     134.643   Cond. No.                     6.67e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.67e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [58]:
# 3: 
modelo = "renda ~ id_cliente + idade + qtd_filhos + qt_pessoas_residencia"
md = smf.ols(modelo, data = df)
reg = md.fit_regularized(method = "elastic_net"
                        , refit = True
                        , L1_wt = 1
                        , alpha = 0.1)
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     13.93
Date:                Fri, 16 Feb 2024   Prob (F-statistic):           1.31e-13
Time:                        02:00:39   Log-Likelihood:            -1.5655e+05
No. Observations:               15000   AIC:                         3.131e+05
Df Residuals:                   14995   BIC:                         3.132e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept              2630.2191    442.985      5.937      0.000    1761.914    3498.524
id_cliente               -0.0161      0.014     -1.146      0.252      -0.044       0.011
idade                    29.1542      6.419      4.542      0.000      16.572      41.736
qtd_filhos             -580.9569    199.538     -2.912      0.004    -972.076    -189.838
qt_pessoas_residencia   984.7392    162.404      6.064      0.000     666.408    1303.070
==============================================================================
Omnibus:                    21437.439   Durbin-Watson:                   2.026
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         11006137.536
Skew:                           8.364   Prob(JB):                         0.00
Kurtosis:                     134.643   Cond. No.                     6.67e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.67e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [59]:
# 3: 
print("Em nenhum dos modelos com alteração do Alpha, houve uma mudança no valor do R-SQUARED.")

Em nenhum dos modelos com alteração do Alpha, houve uma mudança no valor do R-SQUARED.


In [62]:
#Rode um modelo stepwise. Avalie o  𝑅2
#  na base de testes. Qual o melhor resultado?
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

variaveis = stepwise_selection(X, y)

print('resulting features:')
print(variaveis)

Add  qt_pessoas_residencia          with p-value 4.5648e-09
#############
['qt_pessoas_residencia']
Add  idade                          with p-value 4.60683e-07
#############
['qt_pessoas_residencia', 'idade']
Add  qtd_filhos                     with p-value 0.00379203
#############
['qt_pessoas_residencia', 'idade', 'qtd_filhos']
#############
['qt_pessoas_residencia', 'idade', 'qtd_filhos']
resulting features:
['qt_pessoas_residencia', 'idade', 'qtd_filhos']


In [63]:
# 5:
print("Não sei dizer qual modelo é melhor porque me parece que eles não estão completos.")

Não sei dizer qual modelo é melhor porque me parece que eles não estão completos.
